# Иерархия холдинга
Пример расчета с учетом иерархии холдинга.

## Описание задачи

Рассчитаем аудиторию холдинга с разбивкой на все входящие в него медиа объекты.
Построение иерархии возможно для всех уровней медиа объектов.

Иерархия:

- Holding-Resource-Product

Общие параметры:
- Период: 1-30 сентября 2021 года
- География: Россия 0+
- Население: 12+
- Тип пользования интернетом: ограничения нет, считаем по всем (Web Desktop, Web Mobile, App Mobile)

Статистики:
- Reach (reach)

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Создаем объекты для работы с API Cross Web
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

## Формируем задание



### Общие параметры для заданий

Для начала зададим общие параметры

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2021-09-01', '2021-09-30')]

# Задаем фильтр по типам пользования интернетом
usetype_filter = [1,2,3]

### Получим ID холдинга
Для построения отчета необходимо получить идентификатор холдинга __Google__.

Для этого воспользуемся методом поиска в медиа справочнике. Работа с медиа справочником подробно описана в ноутбуке [catalogs](catalogs.ipynb). 

Получим идентификатор холдинга Google

In [ ]:
cats.get_holding(holding='Google')

Таким образом, необходимый идентификатор следующий:

- **Google**  holdingId = 109

## Задания

Перейдем к формированию заданий.


### Задание №1. Построение иерархии

In [ ]:
# Задаем название для отображения в DataFrame
project_name = 'hierarchy'

# Задаем фильтр по географии, в нашем случае он не требуется 
geo_filter = None

# Задаем фильтр по демографии, в нашем случае он не требуется
demo_filter = None

# Задаем фильтр по медиа, в нашем случае это ID холдинга Google
mart_filter = 'crossMediaHoldingId = 109'

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "crossMediaHoldingId", "crossMediaResourceId","crossMediaProductId"]

# Указываем список статистик для расчета
statistics = ['reach']

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('media', project_name, date_filter, usetype_filter, geo_filter, demo_filter, mart_filter, slices, statistics)

# Отправляем задание на расчет и ждем выполнения
task_audience = mtask.wait_task(mtask.send_audience_task(task_json))

# Получаем результат
df_hierarchy = mtask.result2table(mtask.get_result(task_audience), project_name)
df_hierarchy

## Сохраняем в Excel

In [ ]:
writer = pd.ExcelWriter(mtask.get_excel_filename('audience-hierarchy'))
df_info = mtask.get_report_info()
df_hierarchy.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()